In [1]:
using Plots, Distributed, SharedArrays, Distributions#, QuantEcon, Interpolations
addprocs(4)
@everywhere using QuantEcon, Interpolations

In [2]:
@everywhere struct Parameters
    α_1::Float64
    α_2::Float64
    γ_h::Float64
    intercept::Float64
    age1::Float64
    age2::Float64
    age3::Float64
    child::Float64
    interact::Float64
    #ρ::Float64
    #σ::Float64
    gp_ν_s::Int64
    grid_ν_s::Vector{Float64}
    gp_ν_c::Int64
    grid_ν_c::Vector{Float64}
    markov::Matrix{Float64}
    aux_ages::Array{Int64}
    aux_ν::Array{Float64}
    #int_ν_j = interpolate(knots, aux_ν, Gridded(Linear()))
    #min_h::Float64
    #max_h::Float64
    gp_h::Int64
    grid_h::Vector{Float64}
    gp_n::Int64
    grid_n::Vector{Float64}
    starting_age::Int64
    J::Int64
    β::Float64
    θ::Array{Float64,2}
    Δ::Float64
    μ_h::Float64
    σ_h::Float64
    μ_v_c::Float64
    hours_factor::Float64

    function Parameters(α_1 = 0.351;
                        α_2 = 0.379,
                        γ_h = 1.0,
                        intercept = 0.196,
                        age1 = 3.32,
                        age2 = -0.086,
                        age3 = 0.0007,
                        child = -6.81,
                        interact = 0.133,
                        markov,
                        grid_ν_s,
                        gp_ν_s = 7,
                        μ_ν_c = 0.7,
                        gp_ν_c = 7,
                        aux_ages = [17, 20, 25, 30, 40, 50, 55, 60, 65],
                        aux_ν = [8.0, 1.12, 0.42, 0.29, 0.25, 0.24, 0.25, 0.34, 1.6],
                        min_h = 0.0001,
                        max_h = 200,
                        gp_h = 101,
                        gp_n = 5,
                        starting_age = 17,
                        β = 0.99,
                        Δ = 3.0,
                        μ_h = 0.728,
                        σ_h = 0.233,
                        μ_v_c = 0.7)
        grid_h = collect(range(min_h, stop = max_h, length=gp_h))
        grid_n = collect(range(0, stop = gp_n, length = gp_n))
        J = 4*(65-starting_age)
        percentile_ν_c = 0.9 # set percentile
        max_ν_c = -log(1-percentile_ν_c)/μ_ν_c
        grid_ν_c = collect(range(0, stop = max_ν_c, length=gp_ν_c))
        if starting_age==17 ##Non-college
            aux1=[fill(0.0269,5*4); fill(0.0265,5*4); fill(0.0265,5*4); fill(0.0090,8*4); fill(0.0,25*4)]
            aux2=[1.0 1.44 0.76 fill(0.76,gp_n-3)']
            θ = aux1*aux2
            hours_factor = 44.2/40.5 #from table 1
        elseif starting_age==20 ## College
            aux1=[fill(0.0082,5*4); fill(0.0210,5*4); fill(0.0259,5*4); fill(0.0086,5*4); fill(0.0,25*4)]
            aux2=[1.0 2.66 0.76 fill(1.27,gp_n-3)']
            θ = aux1*aux2
            hours_factor = 46.2/42.7 #from table 1
        end
        new(α_1, α_2, γ_h, intercept, age1, age2, age3, child, interact, gp_ν_s, grid_ν_s, gp_ν_c, grid_ν_c, markov, aux_ages, aux_ν, gp_h, grid_h, gp_n, grid_n, starting_age, J, β, θ, Δ, μ_h, σ_h, μ_v_c, hours_factor)
    end
end

In [3]:
@everywhere function c(j,h,P::Parameters)
    return (P.α_1+j^(P.α_2))*h^(P.γ_h)
end
@everywhere function HCA(c,V_hat,V_hat_delta) #Optimal human capital accumulation (HCA)
    e = max(1-c/(V_hat_delta-V_hat),0) #closed form solution of optimal effort
    if e>1
        error("outside of definition")
    end
    return (c*log(1-e)+e*V_hat+(1-e)*V_hat_delta, e)
end

@everywhere function l(j,n,P::Parameters) #working hours
    if j < 40
        P.intercept+P.age1*j+P.age2*j^2+P.age3*j^3+P.child*n+P.interact*n*j
    else#working hours reduction becomes obsolete after age 40
        P.intercept+P.age1*j+P.age2*j^2+P.age3*j^3
    end
end

@everywhere function W(j,h,n,ν,VV_hat,VV_hat_delta,ll,P::Parameters) #Value of working
    cc = c(j,h,P)
    (opt_HCA,e) = HCA(cc,VV_hat,VV_hat_delta)
    uu = h*ν
    aux_val = h*ll + (1-ll)*uu + P.γ_h*log(1+n) + opt_HCA
    return (aux_val, e)
end

@everywhere function H(j,h,n,ν,γ_n,VV_hat) #Value of staying at home
    uu = h*ν
    return uu + γ_n*log(1+n)+VV_hat
end

@everywhere function V(j,h,n,P::Parameters) #Value of optimal labor decision
    max(H(j,h,n,P::Parameters),V(j,h,n,P::Parameters))
end

In [4]:
function male_policy(P::Parameters)
    #Value functions
    W_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s) #Working
    H_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s) #Staying at home
    V_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s) #Optimal labor decision
    #policy functions
    labor_policy = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s) #labor decision
    effort_policy = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s) # effort decision
    #initialisation of remaining variables
    age = SharedArray{Float64}(1)
    ll = SharedArray{Float64}(1)
    itp_h = SharedArray{Float64}(P.gp_h,P.gp_ν_s)
    etp_h = SharedArray{Float64}(P.gp_h,P.gp_ν_s)
    knots = (P.aux_ages,)
    int_ν_j = interpolate(knots, P.aux_ν, Gridded(Linear()))

    #Last period
    ll = P.hours_factor*l(P.J,0,P) #scale working hours up by exogenous factor
    age = P.starting_age+P.J/4
    for (ind_h,h) in enumerate(P.grid_h)
        for (ind_ν, ν_s) in enumerate(P.grid_ν_s)
            ν = int_ν_j[age]*ν_s
            (W_value[ind_h,P.J,ind_ν], effort_policy[ind_h, P.J,ind_ν]) = W(P.J,h,0,ν,10^(-8),10^(-6),ll,P)
            H_value[ind_h,P.J,ind_ν]                                  = H(P.J,h,0,ν,P.γ_h,10^(-8))
            V_value[ind_h,P.J,ind_ν]                                  = max.(W_value[ind_h,P.J,ind_ν],H_value[ind_h,P.J,ind_ν])
            labor_policy[ind_h,P.J,ind_ν]                             = 1*(W_value[ind_h,P.J,ind_ν]>H_value[ind_h,P.J,ind_ν])
        end
    end
    #Backwards induction
    for j in P.J-1:-1:1
        age = P.starting_age+j/4
        ll = P.hours_factor*l(j,0,P) #scale working hours up by exogenous factor
        println(age)
        itp_h = interpolate((P.grid_h,P.grid_ν_s),V_value[:,j+1,:],Gridded(Linear()))
        etp_h = extrapolate(itp_h, Line())
        @sync @distributed for ind in 1:(P.gp_h*P.gp_ν_s)
            ind_ν = convert(Int, ceil(ind/P.gp_h))
            ind_h = convert(Int, floor(mod(ind-0.05, P.gp_h))+1)
            h = P.grid_h[ind_h]
            h_delta = h*(1+(P.Δ/100))
            ν_s = P.grid_ν_s[ind_ν]
            contval = 0
            contval_delta = 0
            for (ind_ν_2, ν_s_2) in enumerate(P.grid_ν_s)
                contval       = contval      + P.markov[ind_ν,ind_ν_2]*etp_h[h,ν_s_2]
                contval_delta = contval_delta+ P.markov[ind_ν,ind_ν_2]*etp_h[h_delta,ν_s_2]
            end
            ν = int_ν_j[age]*ν_s
            (W_value[ind_h,j,ind_ν], effort_policy[ind_h, j,ind_ν]) = W(j, h, 0, ν, contval, contval_delta, ll, P)
            H_value[ind_h,j,ind_ν] = H(j,h,0,ν,P.γ_h, contval)
            V_value[ind_h,j,ind_ν] = max.(W_value[ind_h,j,ind_ν],H_value[ind_h,j,ind_ν])
            labor_policy[ind_h,j,ind_ν] = 1*(W_value[ind_h,j,ind_ν]>H_value[ind_h,j,ind_ν])
        end
    end
    return (W_value, H_value, V_value, labor_policy, effort_policy)
end

male_policy (generic function with 1 method)

In [5]:
function female_policy(P::Parameters)
    #Value functions
    W_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #Working
    H_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #Staying at home
    V_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #Optimal labor decision
    B_value = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #Continuation value
    #Policy functions
    labor_policy = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #labor decision
    effort_policy = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #effort decision
    child_policy = SharedArray{Float64}(P.gp_h, P.J, P.gp_ν_s, P.gp_ν_c, P.gp_n) #fertility decision
    #initialisation of remaining variables
    age = SharedArray{Float64}(1)
    ll = SharedArray{Float64}(1)
    itp_h = SharedArray{Float64}(P.gp_h,P.gp_ν_s, P.gp_ν_c)
    etp_h = SharedArray{Float64}(P.gp_h,P.gp_ν_s, P.gp_ν_c)
    knots = (P.aux_ages,)
    int_ν_j = interpolate(knots, P.aux_ν, Gridded(Linear()))

    #Last period
    age = P.starting_age+P.J/4
    for (ind_n,n) in enumerate(P.grid_n)
        ll = l(P.J,n,P)
        for (ind_h,h) in enumerate(P.grid_h)
            for (ind_ν, ν_s) in enumerate(P.grid_ν_s)
                ν = int_ν_j[age]*ν_s
                (WW, ee)                                              = W(P.J,h,n,ν,10^(-8),10^(-6),ll,P)
                HH                                                    = H(P.J,h,n,ν,P.γ_h,10^(-8))
                W_value[ind_h,P.J,ind_ν,:, ind_n]                     = ones(1,P.gp_ν_c)*WW
                effort_policy[ind_h, P.J,ind_ν,:, ind_n]              = ones(1,P.gp_ν_c)*ee
                H_value[ind_h,P.J,ind_ν,:, ind_n]                     = ones(1,P.gp_ν_c)*HH
                V_value[ind_h,P.J,ind_ν,:, ind_n]                     = ones(1,P.gp_ν_c)*max.(WW,HH)
                B_value[ind_h,P.J,ind_ν,:,ind_n]                      = ones(1,P.gp_ν_c)*max.(WW,HH)
                labor_policy[ind_h,P.J,ind_ν,:, ind_n]                = ones(1,P.gp_ν_c)*(WW>HH)
            end
        end
    end
    #Backwards induction until 40 years -> irrelevant child decision
    for j in P.J-1:-1:((40-P.starting_age)*4)
        age = P.starting_age+j/4
        println(age)
        itp_h = interpolate((P.grid_h,P.grid_ν_s,P.grid_ν_c,P.grid_n),B_value[:,j+1,:,:,:],Gridded(Linear()))
        etp_h = extrapolate(itp_h, Line())
        @sync @distributed for ind in 1:(P.gp_h*P.gp_ν_s)
            ind_ν_s = convert(Int, ceil(ind/P.gp_h))
            ind_h = convert(Int, floor(mod(ind-0.05, P.gp_h))+1)
            h = P.grid_h[ind_h]
            h_delta = h*(1+(P.Δ/100))
            ν_s = P.grid_ν_s[ind_ν_s]
            ν = int_ν_j[age]*(ν_s+0)
            for (ind_n,n) in enumerate(P.grid_n)
                ll = l(j,n,P)
                contval = 0
                contval_delta = 0
                for (ind_ν_2, ν_s_2) in enumerate(P.grid_ν_s)
                    contval       = contval      + P.markov[ind_ν_s,ind_ν_2]*etp_h[h,ν_s_2,0,n]
                    contval_delta = contval_delta+ P.markov[ind_ν_s,ind_ν_2]*etp_h[h_delta,ν_s_2,0,n]
                end
                (WW, ee)                                        = W(j, h, 0, ν, contval, contval_delta, ll, P)
                HH                                              = H(j,h,0,ν,P.γ_h, contval)
                W_value[ind_h,j,ind_ν_s,:, ind_n]                 = ones(1,P.gp_ν_c)*WW
                effort_policy[ind_h, j,ind_ν_s,:, ind_n]          = ones(1,P.gp_ν_c)*ee
                H_value[ind_h,j,ind_ν_s,:, ind_n]                 = ones(1,P.gp_ν_c)*HH
                V_value[ind_h,j,ind_ν_s,:, ind_n]                 = ones(1,P.gp_ν_c)*max.(WW,HH)
                B_value[ind_h,j,ind_ν_s,:,ind_n]                  = ones(1,P.gp_ν_c)*max.(WW,HH)
                labor_policy[ind_h,j,ind_ν_s,:, ind_n]            = ones(1,P.gp_ν_c)*(WW>HH)
            end
        end
    end
    #Backwards induction before 40 years -> relevant child decision
    for j in ((40-P.starting_age)*4-1):-1:1
        age = P.starting_age+j/4
        println(age)
        itp_h = interpolate((P.grid_h,P.grid_ν_s,P.grid_ν_c,P.grid_n),B_value[:,j+1,:,:,:],Gridded(Linear()))
        etp_h = extrapolate(itp_h, Line())
        @sync @distributed for ind in 1:(P.gp_h*P.gp_ν_s)
            ind_ν_s = convert(Int, ceil(ind/P.gp_h))
            ind_h = convert(Int, floor(mod(ind-0.05, P.gp_h))+1)
            h = P.grid_h[ind_h]
            h_delta = h*(1+(P.Δ/100))
            ν_s = P.grid_ν_s[ind_ν_s]
            for (ind_n,n) in enumerate(P.grid_n)
                ll = l(j,n,P)
                for (ind_ν_c, ν_c) in enumerate(P.grid_ν_c)
                    contval = 0
                    contval_delta = 0
                    for (ind_ν_2, ν_s_2) in enumerate(P.grid_ν_s)
                        contval       = contval      + P.markov[ind_ν_s,ind_ν_2]*etp_h[h,ν_s_2,ν_c,n]
                        contval_delta = contval_delta+ P.markov[ind_ν_s,ind_ν_2]*etp_h[h_delta,ν_s_2,ν_c,n]
                    end
                    ν = int_ν_j[age]*(ν_s+ν_c)
                    (W_value[ind_h,j,ind_ν_s,ind_ν_c,ind_n], effort_policy[ind_h, j,ind_ν_s,ind_ν_c, ind_n]) = W(j, h, 0, ν, contval, contval_delta, ll, P)
                    H_value[ind_h,j,ind_ν_s,ind_ν_c, ind_n] = H(j,h,0,ν,P.γ_h, contval)
                    V_value[ind_h,j,ind_ν_s,ind_ν_c, ind_n] = max.(W_value[ind_h,j,ind_ν_s,ind_ν_c, ind_n],H_value[ind_h,j,ind_ν_s,ind_ν_c, ind_n])
                    labor_policy[ind_h,j,ind_ν_s,ind_ν_c, ind_n] = 1*(W_value[ind_h,j,ind_ν_s,ind_ν_c, ind_n]>H_value[ind_h,j,ind_ν_s,ind_ν_c, ind_n])
                end
            end
        end
        @sync @distributed for ind in 1:((P.gp_n-1)*P.gp_ν_c)
            ind_ν_c = convert(Int, ceil(ind/(P.gp_n-1)))
            ind_n = convert(Int, floor(mod(ind-0.05, (P.gp_n-1)))+1)
            child_policy[:,j,:,ind_ν_c, ind_n] = 1*(V_value[:,j,:,ind_ν_c, ind_n+1] .> V_value[:,j,:,ind_ν_c, ind_n])
            B_value[:,j,:,ind_ν_c,ind_n] = P.θ[j,ind_n]*max.(V_value[:,j,:,ind_ν_c,ind_n+1], V_value[:,j,:,1,ind_n])+(1-P.θ[j,ind_n])*V_value[:,j,:,1, ind_n]
        end
    end
    return (W_value, H_value, V_value, B_value, labor_policy, effort_policy, child_policy)
end

female_policy (generic function with 1 method)

In [9]:
ρ = 0.76
σ = 0.79
gp_ν_s = 7
m_non_c = QuantEcon.tauchen(gp_ν_s,ρ,0.79,0,1)
mmpp_non_c = m_non_c.p
@everywhere mp_non_c = $mmpp_non_c
expMM_non_c = exp.(collect(m_non_c.state_values))
@everywhere expM_non_c = $expMM_non_c

P_nonc = Parameters(0.351, α_2 = 0.379, γ_h = 1.0, intercept = 0.196, age1 = 3.32, age2 = -0.086, age3 = 0.0007, child = -6.81, interact = 0.133, markov = mp_non_c, grid_ν_s = expM_non_c, gp_ν_s = 7,
                aux_ages = [17, 20, 25, 30, 40, 50, 55, 60, 65], aux_ν = [8.0, 1.12, 0.42, 0.29, 0.25, 0.24, 0.25, 0.34, 1.6],
                    min_h = 0.0001, max_h = 200, gp_h = 101, gp_n = 5, starting_age = 17, β = 0.99, Δ = 3.0, μ_h = 0.728, σ_h = 0.233, μ_v_c = 0.7)

Parameters(0.351, 0.379, 1.0, 0.196, 3.32, -0.086, 0.0007, -6.81, 0.133, 7, [0.296553, 0.444701, 0.666859, 1.0, 1.49957, 2.2487, 3.37208], 7, [0.0, 0.548235, 1.09647, 1.6447, 2.19294, 2.74117, 3.28941], [0.455081 0.200358 … 0.0191578 0.00711168; 0.307614 0.196478 … 0.0410885 0.0196142; … ; 0.0196142 0.0410885 … 0.196478 0.307614; 0.00711168 0.0191578 … 0.200358 0.455081], [17, 20, 25, 30, 40, 50, 55, 60, 65], [8.0, 1.12, 0.42, 0.29, 0.25, 0.24, 0.25, 0.34, 1.6], 101, [0.0001, 2.0001, 4.0001, 6.0001, 8.0001, 10.0001, 12.0001, 14.0001, 16.0001, 18.0001  …  182.0, 184.0, 186.0, 188.0, 190.0, 192.0, 194.0, 196.0, 198.0, 200.0], 5, [0.0, 1.25, 2.5, 3.75, 5.0], 17, 192, 0.99, [0.0269 0.038736 … 0.020444 0.020444; 0.0269 0.038736 … 0.020444 0.020444; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], 3.0, 0.728, 0.233, 0.7, 1.0913580246913581)

In [8]:
m_c = QuantEcon.tauchen(gp_ν_s,ρ,1.345,0,1)
mmpp_c = m_c.p
@everywhere mp_c = $mmpp_c
expMM_c = exp.(collect(m_c.state_values))
@everywhere expM_c = $expMM_c

P_c = Parameters(-0.31, α_2 = 0.457, γ_h = 1.0, intercept = -147.8, age1 = 16.09, age2 = -0.44, age3 = 0.0039, child = -12.56, interact = 0.239, markov = mp_c, grid_ν_s = expM_c, gp_ν_s = 7,
                aux_ages = [20, 21, 23, 25, 30, 40, 50, 60, 65], aux_ν = [35.10, 5.0, 0.35, 0.38, 0.07, 0.05, 0.05, 0.20, 0.86],
                    min_h = 0.0001, max_h = 200, gp_h = 101, gp_n = 5, starting_age = 20, β = 0.99, Δ = 4.15, μ_h = 0.976, σ_h = 0.395, μ_v_c = 4.1)

Parameters(-0.31, 0.457, 1.0, -147.8, 16.09, -0.44, 0.0039, -12.56, 0.239, 7, [0.126252, 0.251667, 0.501664, 1.0, 1.99337, 3.97351, 7.92067], 7, [0.0, 0.548235, 1.09647, 1.6447, 2.19294, 2.74117, 3.28941], [0.455081 0.200358 … 0.0191578 0.00711168; 0.307614 0.196478 … 0.0410885 0.0196142; … ; 0.0196142 0.0410885 … 0.196478 0.307614; 0.00711168 0.0191578 … 0.200358 0.455081], [20, 21, 23, 25, 30, 40, 50, 60, 65], [35.1, 5.0, 0.35, 0.38, 0.07, 0.05, 0.05, 0.2, 0.86], 101, [0.0001, 2.0001, 4.0001, 6.0001, 8.0001, 10.0001, 12.0001, 14.0001, 16.0001, 18.0001  …  182.0, 184.0, 186.0, 188.0, 190.0, 192.0, 194.0, 196.0, 198.0, 200.0], 5, [0.0, 1.25, 2.5, 3.75, 5.0], 20, 180, 0.99, [0.0082 0.021812 … 0.010414 0.010414; 0.0082 0.021812 … 0.010414 0.010414; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], 4.15, 0.976, 0.395, 4.1, 1.0819672131147542)

In [10]:
@time (W_value_m_c, H_value_m_c, V_value_m_c, labor_policy_m_c, effort_policy_m_c) = male_policy(P_c)
@time (W_value_m_nonc, H_value_m_nonc, V_value_m_nonc, labor_policy_m_nonc, effort_policy_m_nonc) = male_policy(P_nonc)

┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = male_policy(::Parameters) at In[4]:22
└ @ Main ./In[4]:22


64.75


┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = (::getfield(Serialization.__deserialized_types__, Symbol("##5#6")){Parameters,SharedArrays.SharedArray{Float64,3},SharedArrays.SharedArray{Float64,3},SharedArrays.SharedArray{Float64,3},SharedArrays.SharedArray{Float64,3},SharedArrays.SharedArray{Float64,3},Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Int64})(::UnitRange{Int64}, ::Int64, ::Int64) at In[4]:45
└ @ Main ./In[4]:45
┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = (::getfield(Serialization.__deserialized_types__, Symbol("##5#6")){Parameters,SharedArrays.SharedArray{Float64,3},SharedArrays.SharedArray{Float64,3},SharedArrays.SharedArray{Float64,3},SharedArrays.SharedArray{Float64,3},SharedArrays.SharedArray{Float64,3},Interpolations.GriddedInt

64.5
64.25
64.0
63.75
63.5
63.25
63.0
62.75
62.5
62.25
62.0
61.75
61.5
61.25
61.0
60.75
60.5
60.25
60.0
59.75
59.5
59.25
59.0
58.75
58.5
58.25
58.0
57.75
57.5
57.25
57.0
56.75
56.5
56.25
56.0
55.75
55.5
55.25
55.0
54.75
54.5
54.25
54.0
53.75
53.5
53.25
53.0
52.75
52.5
52.25
52.0
51.75
51.5
51.25
51.0
50.75
50.5
50.25
50.0
49.75
49.5
49.25
49.0
48.75
48.5
48.25
48.0
47.75
47.5
47.25
47.0
46.75
46.5
46.25
46.0
45.75
45.5
45.25
45.0
44.75
44.5
44.25
44.0
43.75
43.5
43.25
43.0
42.75
42.5
42.25
42.0
41.75
41.5
41.25
41.0
40.75
40.5
40.25
40.0
39.75
39.5
39.25
39.0
38.75
38.5
38.25
38.0
37.75
37.5
37.25
37.0
36.75
36.5
36.25
36.0
35.75
35.5
35.25
35.0
34.75
34.5
34.25
34.0
33.75
33.5
33.25
33.0
32.75
32.5
32.25
32.0
31.75
31.5
31.25
31.0
30.75
30.5
30.25
30.0
29.75
29.5
29.25
29.0
28.75
28.5
28.25
28.0
27.75
27.5
27.25
27.0
26.75
26.5
26.25
26.0
25.75
25.5
25.25
25.0
24.75
24.5
24.25
24.0
23.75
23.5
23.25
23.0
22.75
22.5
22.25
22.0
21.75
21.5
21.25
21.0
20.75
20.5
20.25
 77.718416 seconds (9

([4.62094 4.61969 … 0.219555 0.138948; 92423.5 92398.3 … 4391.3 2779.07; … ; 9.14947e6 9.14698e6 … 4.34717e5 2.75115e5; 9.24189e6 9.23937e6 … 4.39108e5 2.77894e5]

[4.62097 4.61936 … 0.140054 0.07632; 92423.9 92391.8 … 2801.22 1526.46; … ; 9.14952e6 9.14633e6 … 2.77307e5 1.51112e5; 9.24194e6 9.23872e6 … 280108.0 152638.0]

[4.62092 4.6188 … 0.0327504 -0.0175945; 92423.0 92380.6 … 655.033 -351.928; … ; 9.14943e6 9.14522e6 … 64845.1 -34839.1; 9.24185e6 9.2376e6 … 65500.1 -35191.0]

[4.62073 4.61784 … -0.115002 -0.158426; 92419.1 92361.4 … -2300.16 -3168.69; … ; 9.14904e6 9.14333e6 … -2.27705e5 -3.13685e5; 9.24145e6 9.23568e6 … -2.30005e5 -3.16853e5]

[4.62023 4.61623 … -0.324908 -0.369611; 92409.2 92329.2 … -6498.48 -7392.61; … ; 9.14806e6 9.14014e6 … -6.43318e5 -731832.0; 9.24047e6 9.23246e6 … -6.49816e5 -7.39224e5]

[4.61924 4.61359 … -0.6298 -0.686298; 92389.4 92276.3 … -12596.6 -13726.7; … ; 9.1461e6 9.1349e6 … -1.247e6 -1.35887e6; 9.23848e6 9.22718e6 … -1.2596e6 -1.3726e6]

[4.61745

In [11]:
@time (W_value_f_c, H_value_f_c, V_value_f_c, B_value_f_c, labor_policy_f_c, effort_policy_f_c, child_policy_f_c) = female_policy(P_c)

┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = female_policy(::Parameters) at In[5]:25
└ @ Main ./In[5]:25


64.75


┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = macro expansion at In[5]:49 [inlined]
└ @ Core ./In[5]:49
┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = macro expansion at In[5]:49 [inlined]
└ @ Core ./In[5]:49
┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = macro expansion at In[5]:49 [inlined]
└ @ Core ./In[5]:49
┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = macro expansion at In[5]:49 [inlined]
└ @ Core ./In[5]:49
┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = (::getfield(Serialization.__deserialized_types__

64.5
64.25
64.0
63.75
63.5
63.25
63.0
62.75
62.5
62.25
62.0
61.75
61.5
61.25
61.0
60.75
60.5
60.25
60.0
59.75
59.5
59.25
59.0
58.75
58.5
58.25
58.0
57.75
57.5
57.25
57.0
56.75
56.5
56.25
56.0
55.75
55.5
55.25
55.0
54.75
54.5
54.25
54.0
53.75
53.5
53.25
53.0
52.75
52.5
52.25
52.0
51.75
51.5
51.25
51.0
50.75
50.5
50.25
50.0
49.75
49.5
49.25
49.0
48.75
48.5
48.25
48.0
47.75
47.5
47.25
47.0
46.75
46.5
46.25
46.0
45.75
45.5
45.25
45.0
44.75
44.5
44.25
44.0
43.75
43.5
43.25
43.0
42.75
42.5
42.25
42.0
41.75
41.5
41.25
41.0
40.75
40.5
40.25
40.0
39.75


┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = (::getfield(Serialization.__deserialized_types__, Symbol("##8#11")){Parameters,SharedArrays.SharedArray{Float64,5},SharedArrays.SharedArray{Float64,5},SharedArrays.SharedArray{Float64,5},SharedArrays.SharedArray{Float64,5},SharedArrays.SharedArray{Float64,5},Interpolations.GriddedInterpolation{Float64,1,Float64,Gridded{Linear},Tuple{Array{Int64,1}}},Int64})(::UnitRange{Int64}, ::Int64, ::Int64) at In[5]:87
└ @ Main ./In[5]:87
┌ Warning: `getindex(itp::AbstractInterpolation{T, N}, i::Vararg{Number, N}) where {T, N}` is deprecated, use `itp(i...)` instead.
│   caller = (::getfield(Serialization.__deserialized_types__, Symbol("##8#11")){Parameters,SharedArrays.SharedArray{Float64,5},SharedArrays.SharedArray{Float64,5},SharedArrays.SharedArray{Float64,5},SharedArrays.SharedArray{Float64,5},SharedArrays.SharedArray{Float64,5},Interpolations.GriddedI

39.5
39.25
39.0
38.75
38.5
38.25
38.0
37.75
37.5
37.25
37.0
36.75
36.5
36.25
36.0
35.75
35.5
35.25
35.0
34.75
34.5
34.25
34.0
33.75
33.5
33.25
33.0
32.75
32.5
32.25
32.0
31.75
31.5
31.25
31.0
30.75
30.5
30.25
30.0
29.75
29.5
29.25
29.0
28.75
28.5
28.25
28.0
27.75
27.5
27.25
27.0
26.75
26.5
26.25
26.0
25.75
25.5
25.25
25.0
24.75
24.5
24.25
24.0
23.75
23.5
23.25
23.0
22.75
22.5
22.25
22.0
21.75
21.5
21.25
21.0
20.75
20.5
20.25
1184.250068 seconds (4.55 M allocations: 295.901 MiB, 0.01% gc time)


([25.5252 25.2828 … 1.73621 1.00172; 4.92698e5 4.87932e5 … 34726.0 20035.4; … ; 4.87747e7 4.83029e7 … 3.4377e6 1.98341e6; 4.92673e7 4.87908e7 … 3.47242e6 2.00344e6]

[25.7028 25.3802 … 1.49869 0.880532; 4.9625e5 489880.0 … 29975.2 17611.5; … ; 4.91263e7 4.84957e7 … 2.9674e6 1.74345e6; 4.96225e7 4.89855e7 … 2.99737e6 1.76106e6]

[25.979 25.5347 … 1.1324 0.638957; 5.01776e5 4.92971e5 … 22649.0 12779.7; … ; 4.96733e7 4.88016e7 … 2.24214e6 1.26513e6; 5.0175e7 4.92946e7 … 2.26479e6 1.27791e6]

[26.4041 25.7785 … 0.540985 0.157408; 5.10276e5 4.97846e5 … 10820.2 3148.29; … ; 5.05148e7 4.92842e7 … 1.07115e6 3.11665e5; 5.10251e7 4.9782e7 … 1.08197e6 3.14814e5]

[27.0634 26.1676 … -0.483789 -0.802496; 5.23464e5 5.05629e5 … -9676.28 -16050.7; … ; 5.18203e7 5.00547e7 … -9.57904e5 -1.58894e6; 5.23438e7 5.05603e7 … -9.6758e5 -1.60499e6]

[28.1192 26.8094 … -2.37866 -2.71594; 5.44582e5 5.18465e5 … -47575.5 -54321.4; … ; 5.39109e7 5.13254e7 … -4.70974e6 -5.37756e6; 5.44554e7 5.18438e7 … -4.75732e6 -5.

In [13]:
@time (W_value_f_nonc, H_value_f_nonc, V_value_f_nonc, B_value_f_nonc, labor_policy_f_nonc, effort_policy_f_nonc, child_policy_f_nonc) = female_policy(P_nonc)

64.75
64.5
64.25
64.0
63.75
63.5
63.25
63.0
62.75
62.5
62.25
62.0
61.75
61.5
61.25
61.0
60.75
60.5
60.25
60.0
59.75
59.5
59.25
59.0
58.75
58.5
58.25
58.0
57.75
57.5
57.25
57.0
56.75
56.5
56.25
56.0
55.75
55.5
55.25
55.0
54.75
54.5
54.25
54.0
53.75
53.5
53.25
53.0
52.75
52.5
52.25
52.0
51.75
51.5
51.25
51.0
50.75
50.5
50.25
50.0
49.75
49.5
49.25
49.0
48.75
48.5
48.25
48.0
47.75
47.5
47.25
47.0
46.75
46.5
46.25
46.0
45.75
45.5
45.25
45.0
44.75
44.5
44.25
44.0
43.75
43.5
43.25
43.0
42.75
42.5
42.25
42.0
41.75
41.5
41.25
41.0
40.75
40.5
40.25
40.0
39.75
39.5
39.25
39.0
38.75
38.5
38.25
38.0
37.75
37.5
37.25
37.0
36.75
36.5
36.25
36.0
35.75
35.5
35.25
35.0
34.75
34.5
34.25
34.0
33.75
33.5
33.25
33.0
32.75
32.5
32.25
32.0
31.75
31.5
31.25
31.0
30.75
30.5
30.25
30.0
29.75
29.5
29.25
29.0
28.75
28.5
28.25
28.0
27.75
27.5
27.25
27.0
26.75
26.5
26.25
26.0
25.75
25.5
25.25
25.0
24.75
24.5
24.25
24.0
23.75
23.5
23.25
23.0
22.75
22.5
22.25
22.0
21.75
21.5
21.25
21.0
20.75
20.5
20.25
20.0
19.75
19.5

([5.27401 5.2637 … 0.201247 0.12732; 84300.2 84275.6 … 4025.12 2546.51; … ; 8.34525e6 8.34281e6 … 3.98467e5 2.52092e5; 8.42955e6 8.42708e6 … 4.02492e5 2.54639e5]

[5.27407 5.26344 … 0.128425 0.0699373; 84301.4 84270.3 … 2568.62 1398.79; … ; 8.34537e6 8.34229e6 … 2.54281e5 1.38474e5; 8.42966e6 8.42655e6 … 2.56849e5 1.39873e5]

[5.27408 5.26297 … 0.0301351 -0.0161126; 84301.6 84260.9 … 602.724 -322.289; … ; 8.34538e6 8.34136e6 … 59666.8 -31905.0; 8.42968e6 8.42561e6 … 60269.5 -32227.3]

[5.27397 5.26215 … -0.105345 -0.14515; 84299.2 84244.5 … -2107.0 -2903.17; … ; 8.34514e6 8.33973e6 … -2.08583e5 -2.87399e5; 8.42944e6 8.42397e6 … -210690.0 -2.90302e5]

[5.2736 5.26074 … -0.297668 -0.338651; 84291.7 84216.3 … -5953.66 -6773.37; … ; 8.3444e6 8.33695e6 … -5.89383e5 -6.7053e5; 8.42869e6 8.42116e6 … -5.95336e5 -6.77303e5]

[5.2728 5.2584 … -0.577023 -0.628817; 84275.3 84169.6 … -11541.0 -12577.0; … ; 8.34278e6 8.33232e6 … -1.14251e6 -1.24506e6; 8.42705e6 8.41648e6 … -1.15405e6 -1.25764e6]

[5